In [37]:
import numpy as np
import matplotlib.pyplot as plt
from sjoert.stellar import flux2mag,mag2flux
import pandas as pd
import forced_phot as fp #import flux_jy, chi2, gaussian, chi2_peak_finder
import os

param_label_dict = {'sigma_rise':r'$\sigma_{\text{rise}}$','tau_dec':r'$\tau_{\text{dec}}$',
                    'log10_sigma_rise':r'$\sigma_{\text{rise}}$','log10_tau_dec':r'$\tau_{\text{dec}}$',
                    'F_p':r'F$_{\text{peak}}$','peak_g':r'F$_{\text{p,g}}$','peak_r':r'F$_{\text{p,r}}$','MBH':r'$\log_{10}($M$_{\text{BH}}/M_{\odot})$',
                    'dmg':r'$\Delta$m$_g$','dmr':r'$\Delta$m$_r$'}

#==
#Load in the data
dpath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data'
fullcatalogue = pd.read_csv(os.path.join(dpath,'transient_catalogue.txt'))
# catalogue = fullcatalogue[fullcatalogue['in_selection']].iloc[:,:-2]


In [38]:
def find_3_sigma_tau(t,t0,sigma,tau):
    sigma_thresh = t0 - 3 * sigma
    tau__thresh = t0 + 3 * tau
    tmask = np.logical_or((t <= sigma_thresh),(t >= tau__thresh))

    if tmask.sum() >= 30:
        return tmask
    else:
        return None
    
def rms(arr):
    return np.sqrt(np.mean(np.square(arr)))

def optical_df_over_rms(flux,time,t0,sigma,tau,delta_m):
    deltaF = mag2flux(delta_m)
    mask = find_3_sigma_tau(time,t0,sigma,tau)
    if mask is not None:
        f_rms = rms(flux[mask])
        return deltaF / f_rms
    else:
        return None



In [53]:
def gen_paths():
    ztf_ids = fullcatalogue['ZTF_ID'].values
    datapaths = []
    for ztf in ztf_ids:
        year = ztf[3:5]
        datapaths.append(os.path.join(dpath,year,ztf))
    return datapaths
datapaths = gen_paths()

In [55]:
test = [1,2,4]

In [56]:
test_ztf = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\18\ZTF18abjjkeo'
def run_optical_rms(ztf_dir=test_ztf,ztf_name=None):
    if ztf_name == None:
        ztf_name = os.path.split(ztf_dir)[-1]

    full_data_dir = os.path.join(ztf_dir,ztf_name+'_clean_data.txt')
    try:
        params = fullcatalogue[fullcatalogue['ZTF_ID'] == ztf_name][['t_0','sigma_rise','tau_dec','dmg']].values[0]
    except IndexError: #ZTF ID is not in the full catalogue
        return None

    clean_data = pd.read_csv(full_data_dir,
                                 sep='\t',comment='#',
                                 names=['time','flux','flux_unc','zeropoint','filter'])[['time','flux']]
    
    return optical_df_over_rms(clean_data['flux'].values,clean_data['time'].values,*params)

def run_all():
    optical_df_over_rms = []
    for dir in datapaths:
        optical_df_over_rms.append(run_optical_rms(dir))
    
    return optical_df_over_rms

##Run and save to dataframe, overwritin the previous
# optical_df_over_rms = run_all()
# fullcatalogue['opt_df_over_rms'] = optical_df_over_rms
# fullcatalogue.to_csv(os.path.join(dpath,'transient_catalogue.txt'))